In [1]:
import pandas as pd
import numpy as np
import re
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from numbers import Number
import statsmodels.formula.api as smf

raw_data = pd.read_csv('./dataset/Video_Game_Sales_as_of_Jan_2017.csv', usecols=['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Rating'])
raw_data.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.54,76.0,51.0,8.0,324.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.80,3.79,3.29,35.57,82.0,73.0,8.3,712.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.95,3.28,2.95,32.78,80.0,73.0,8.0,193.0,E
4,Pokemon Red/Pokemon Blue,G,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN


## Prepare data

In [2]:
# Years in dataset
# min(prepared_data.Year_of_Release) # 1985
# max(prepared_data.Year_of_Release) # 2016

genres = ['Sports', 'Platform', 'Racing', 'Role-Playing',
          'Puzzle', 'Misc','Shooter', 'Simulation', 'Action',
          'Fighting', 'Adventure','Strategy']

# remove all null columns
raw_data = raw_data.dropna() # original 17 416, without null 7 112

train_set, test_set = train_test_split(raw_data, test_size = 0.2)

def get_decade(row):
    if row['Year_of_Release'] <= 1990:    
        return 1980
    elif row['Year_of_Release'] <= 2000:
        return 1990
    elif row['Year_of_Release'] <= 2010:
        return 2000
    else:
        return 2010

def has_numbers(row):
    if bool(re.search(r'\d', row['Name'])):
        return 1
    else:
        return 0

def label_genres(row,genre):
    if(row['Genre'] == genre):
        return 1
    else:
        return 0
    
def get_filtered_data(data):
    #delete null data
    prepared_data = data.dropna()
    #add Decade column
    prepared_data = prepared_data.copy()
    prepared_data['Decade'] = prepared_data.apply(get_decade,axis=1)
    #add Sequel column
    prepared_data['Sequel'] = prepared_data.apply(has_numbers,axis=1)
    
    for genre in genres:
        prepared_data[genre] = prepared_data.apply(lambda row: label_genres(row,genre), axis=1)
    
    
    return prepared_data

## Linear Regression - EU sales